In [ ]:
#@title نصب کتابخانه ها
import base64

encoded_url = "aHR0cHM6Ly9naXRodWIuY29tL3lhcmFuYmFyemkvYWlnb2xkZW4tYXVkaW8tdG8tdGV4dC5naXQ="
decoded_url = base64.b64decode(encoded_url.encode()).decode()
!pip install git+$decoded_url
! pip install edge_tts
!pip install yt-dlp
!pip install pysrt
!pip install rubberband-cli
!pip install pydub
!sudo apt update && sudo apt install ffmpeg

In [ ]:
#@title اتصال به گوگل درایو
#@markdown #### فقط در صورتی اجرا کنید که ویدیوی مورد نظر در گوگل درایو باشد
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title آپلود ویدیو از سه طریق مختلف
from google.colab import files
from IPython.display import display
import ipywidgets as widgets
import yt_dlp
import os
import shutil

# پاکسازی فایل‌های قبلی
if os.path.exists('input_video.mp4'):
    os.remove('input_video.mp4')
    print("فایل ویدیوی قبلی حذف شد.")

if os.path.exists('audio.wav'):
    os.remove('audio.wav')
    print("فایل صوتی قبلی حذف شد.")

if os.path.exists('audio.srt'):
    os.remove('audio.srt')
    print("فایل زیرنویس قبلی حذف شد.")

if os.path.exists('dubbing_project'):
    shutil.rmtree('dubbing_project')
    print("پوشه پروژه دوبله قبلی حذف شد.")

upload_method = "یوتیوب" #@param ["حافظه داخلی", "یوتیوب", "گوگل درایو"]
#@markdown #### محل وارد کردن لینک ویدیوی یوتیوب
YT_Link = "" #@param {type:"string"}
#@markdown #### محل وارد کردن مسیر گوگل درایو
drive_path = "" #@param {type:"string"}

def process_youtube(url):
    if url.strip():
        # دانلود ویدیو
        video_opts = {
            'format': 'best',
            'outtmpl': 'input_video.mp4',
            'nocheckcertificate': True,
            'ignoreerrors': True,
            'no_warnings': True,
            'quiet': True
        }
        with yt_dlp.YoutubeDL(video_opts) as ydl:
            ydl.download([url])

        # استخراج صدا
        audio_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
            }],
            'outtmpl': 'audio',
            'nocheckcertificate': True,
            'ignoreerrors': True,
            'no_warnings': True,
            'quiet': True
        }
        with yt_dlp.YoutubeDL(audio_opts) as ydl:
            ydl.download([url])
        return True
    return False

if upload_method == "یوتیوب" and YT_Link.strip():
    process_youtube(YT_Link)
    print("ویدیو و فایل صوتی از یوتیوب دانلود شدند.")

elif upload_method == "گوگل درایو" and drive_path.strip():
    os.system(f'cp "{drive_path}" input_video.mp4')
    os.system('ffmpeg -i input_video.mp4 -vn audio.wav')
    print("ویدیو و فایل صوتی از گوگل درایو کپی شدند.")

elif upload_method == "حافظه داخلی":
    print("لطفاً ویدیوی خود را آپلود کنید:")
    uploaded = files.upload()
    video_file = next(iter(uploaded.keys()))
    os.system(f'ffmpeg -i "{video_file}" -vn audio.wav')
    os.rename(video_file, 'input_video.mp4')
    print("ویدیو آپلود و فایل صوتی استخراج شد.")

print("عملیات با موفقیت انجام شد!")

In [ ]:
#@title استخراج متن از فایل صوتی
from google.colab import files
import os

extraction_method = "Whisper" #@param ["Whisper", "آپلود زیرنویس"]

if extraction_method == "Whisper":
    if os.path.exists('audio.wav'):
        !whisper "audio.wav" --model large --output_dir ./ --output_format srt
        os.rename("audio.srt", "audio.srt")  # اطمینان از نام فایل صحیح
    else:
        print("لطفاً ابتدا یک فایل صوتی آپلود کنید.")

else:
    print("لطفاً فایل زیرنویس خود را با فرمت .srt آپلود کنید:")
    uploaded = files.upload()
    subtitle_file = next(iter(uploaded.keys()))
    os.rename(subtitle_file, 'audio.srt')

In [ ]:
#@title ترجمه زیرنویس
import pysrt
import google.generativeai as genai
from tqdm.notebook import tqdm
import time
from tenacity import retry, stop_after_attempt, wait_exponential
from google.colab import files
import os

translation_method = "هوش مصنوعی" #@param ["هوش مصنوعی", "آپلود زیرنویس بصورت دستی"]
source_language = "English (EN)" #@param ["English (EN)", "Persian (FA)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)"]
target_language = "Persian (FA)" #@param ["Persian (FA)", "English (EN)", "German (DE)", "French (FR)", "Italian (IT)", "Spanish (ES)", "Chinese (ZH)", "Korean (KO)", "Russian (RU)", "Arabic (AR)", "Japanese (JA)"]

if translation_method == "هوش مصنوعی":
    # @markdown  **لطفاً کلید خود را در کادر زیر وارد کنید (API KEY)**
    GOOGLE_API_KEY = "" #@param {type:"string"}

    if not GOOGLE_API_KEY:
        print("هشدار: کلید API وارد نشده است.")
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("کلید API با موفقیت پیکربندی شد.")

        filename = '/content/audio.srt'
        output_filename = '/content/audio_translated.srt'

        @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
        def translate_subtitle_with_retry(text):
            model = genai.GenerativeModel('gemini-1.5-flash', safety_settings={
                genai.types.HarmCategory.HARM_CATEGORY_HARASSMENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                genai.types.HarmCategory.HARM_CATEGORY_HATE_SPEECH: genai.types.HarmBlockThreshold.BLOCK_NONE,
                genai.types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: genai.types.HarmBlockThreshold.BLOCK_NONE,
                genai.types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: genai.types.HarmBlockThreshold.BLOCK_NONE,
            })

            # تنظیم پرامپت براساس زبان مقصد
            language_map = {
                "Persian (FA)": "فارسی",
                "English (EN)": "English",
                "German (DE)": "German",
                "French (FR)": "French",
                "Italian (IT)": "Italian",
                "Spanish (ES)": "Spanish",
                "Chinese (ZH)": "Chinese",
                "Korean (KO)": "Korean",
                "Russian (RU)": "Russian",
                "Arabic (AR)": "Arabic",
                "Japanese (JA)": "Japanese"
            }

            target_lang_name = language_map.get(target_language, "English")

            if target_language == "Persian (FA)":
                prompt = f"""دستورالعمل:
                1. فقط متن را به فارسی عامیانه و لحن خودمونی ترجمه کن
                2. هرجا لازمه از نقطه و کاما و علائم نگارشی استفاده کن
                3. اضافه گویی در ترجمه ممنوع
                متن برای ترجمه:
                {text}"""
            else:
                prompt = f"""Instruction:
                1. Please translate the text to {target_lang_name} with the same tone
                2. Use appropriate punctuation where necessary
                3. No additional explanation or text
                Text to translate:
                {text}"""

            response = model.generate_content(prompt)
            time.sleep(3)
            return response.text

        try:
            subs = pysrt.open(filename)
            with tqdm(total=len(subs), desc="ترجمه زیرنویس") as pbar:
                for i, sub in enumerate(subs):
                    sub.text = translate_subtitle_with_retry(sub.text)
                    pbar.update(1)
            subs.save(output_filename, encoding='utf-8')
            os.rename(output_filename, 'audio_fa.srt')  # برای سازگاری با کد قبلی
            print(f"\nترجمه از {source_language} به {target_language} با موفقیت به پایان رسید!")
        except Exception as e:
            print(f"خطا در خواندن فایل زیرنویس: {str(e)}")

else:
    print("لطفاً فایل زیرنویس ترجمه شده خود را آپلود کنید:")
    uploaded = files.upload()
    subtitle_file = next(iter(uploaded.keys()))
    os.rename(subtitle_file, 'audio_fa.srt')
    print("زیرنویس ترجمه شده با موفقیت آپلود شد.")

In [ ]:

#@title ساخت سگمنت‌های صوتی با زمان‌بندی دقیق
import edge_tts
import asyncio
import pysrt
import os
import subprocess

os.makedirs('dubbing_project/dubbed_segments', exist_ok=True)

voice_choice = "دلارا (FA)" #@param ["فرید (FA)", "دلارا (FA)", "Jenny (EN)", "Guy (EN)", "Katja (DE)", "Conrad (DE)", "Elvira (ES)", "Alvaro (ES)", "Denise (FR)", "Henri (FR)", "Nanami (JA)", "Keita (JA)", "SunHi (KO)", "InJoon (KO)", "Xiaoxiao (ZH)", "Yunyang (ZH)", "Svetlana (RU)", "Dmitry (RU)", "Amina (AR)", "Hamed (AR)", "Isabella (IT)", "Diego (IT)"]

# تعریف گوینده‌ها برای همه زبان‌ها
VOICE_MAP = {
    # فارسی
    "فرید (FA)": "fa-IR-FaridNeural",
    "دلارا (FA)": "fa-IR-DilaraNeural",
    # انگلیسی
    "Jenny (EN)": "en-US-JennyNeural",
    "Guy (EN)": "en-US-GuyNeural",
    # آلمانی
    "Katja (DE)": "de-DE-KatjaNeural",
    "Conrad (DE)": "de-DE-ConradNeural",
    # فرانسوی
    "Denise (FR)": "fr-FR-DeniseNeural",
    "Henri (FR)": "fr-FR-HenriNeural",
    # ایتالیایی
    "Isabella (IT)": "it-IT-IsabellaNeural",
    "Diego (IT)": "it-IT-DiegoNeural",
    # اسپانیایی
    "Elvira (ES)": "es-ES-ElviraNeural",
    "Alvaro (ES)": "es-ES-AlvaroNeural",
    # چینی
    "Xiaoxiao (ZH)": "zh-CN-XiaoxiaoNeural",
    "Yunyang (ZH)": "zh-CN-YunyangNeural",
    # کره‌ای
    "SunHi (KO)": "ko-KR-SunHiNeural",
    "InJoon (KO)": "ko-KR-InJoonNeural",
    # روسی
    "Svetlana (RU)": "ru-RU-SvetlanaNeural",
    "Dmitry (RU)": "ru-RU-DmitryNeural",
    # عربی
    "Amina (AR)": "ar-EG-AminaNeural",
    "Hamed (AR)": "ar-EG-HamedNeural",
    # ژاپنی
    "Nanami (JA)": "ja-JP-NanamiNeural",
    "Keita (JA)": "ja-JP-KeitaNeural"
}

async def generate_speech():
    subs = pysrt.open('/content/audio_fa.srt')

    selected_voice = VOICE_MAP.get(voice_choice)
    if not selected_voice:
        print(f"گوینده انتخاب شده '{voice_choice}' در لیست موجود نیست. لطفاً یک گوینده معتبر انتخاب کنید.")
        return

    print(f"شروع تولید سگمنت‌های صوتی با صدای {voice_choice} ({selected_voice})...")

    for i, sub in enumerate(subs):
        # محاسبه مدت زمان دقیق
        start_time = sub.start.seconds + sub.start.milliseconds/1000
        end_time = sub.end.seconds + sub.end.milliseconds/1000
        target_duration = end_time - start_time

        # تولید صدا با Edge TTS
        communicate = edge_tts.Communicate(sub.text, selected_voice)
        await communicate.save(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3")

        # استفاده از ffmpeg برای تبدیل به wav (برای اطمینان از فرمت صحیح)
        subprocess.run([
            'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_{i+1}.mp3",
            '-y', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"
        ])

        # محاسبه مدت زمان اصلی فایل صوتی
        result = subprocess.run([
            'ffprobe', '-v', 'error', '-show_entries', 'format=duration',
            '-of', 'default=noprint_wrappers=1:nokey=1',
            f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"
        ], capture_output=True, text=True)

        original_duration = float(result.stdout.strip())

        # محاسبه ضریب سرعت برای رسیدن به مدت زمان هدف
        speed_factor = original_duration / target_duration

        # استفاده از فیلتر rubberband در ffmpeg برای تغییر مدت زمان با حفظ کیفیت
        subprocess.run([
            'ffmpeg', '-i', f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav",
            '-filter:a', f'rubberband=tempo={speed_factor}',
            '-y', f"dubbing_project/dubbed_segments/dub_{i+1}.wav"
        ])

        # پاکسازی فایل‌های موقت
        if os.path.exists(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3"):
            os.remove(f"dubbing_project/dubbed_segments/temp_{i+1}.mp3")
        if os.path.exists(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav"):
            os.remove(f"dubbing_project/dubbed_segments/temp_wav_{i+1}.wav")

        print(f"بخش {i+1} از {len(subs)} ساخته شد (زمان هدف: {target_duration:.2f}s، زمان اصلی: {original_duration:.2f}s)")

# نصب rubberband-cli اگر نصب نیست
!apt-get install -y rubberband-cli

await generate_speech()
print(f"تمام سگمنت‌های صوتی با صدای {voice_choice} با زمان‌بندی دقیق ساخته شدند!")

In [ ]:

#@title شروع عملیات
import subprocess
import pysrt
import os
from datetime import timedelta

#@markdown #### تنظیمات صدای اصلی

#@markdown آیا می‌خواهید صدای اصلی ویدیو حفظ شود؟
keep_original_audio = False #@param {type:"boolean"}

#@markdown میزان صدای اصلی ویدیو (فقط در صورت فعال بودن گزینه بالا)
original_audio_volume = 0.05 #@param {type:"slider", min:0, max:1, step:0.005}

# بررسی وجود فایل‌های مورد نیاز
if not os.path.exists('input_video.mp4'):
    print("لطفاً نام ویدیوی ورودی را به input_video.mp4 تغییر دهید")
    exit()

# خواندن و پردازش فایل زیرنویس
subs = pysrt.open('/content/audio_fa.srt')

# تنظیم فیلتر complex برای ترکیب صدا
if keep_original_audio:
    # اگر کاربر خواسته صدای اصلی حفظ شود، از volume برای تنظیم میزان صدا استفاده می‌کنیم
    filter_complex = f"[0:a]volume={original_audio_volume}[original_audio];"
else:
    # در غیر این صورت صدای اصلی را کاملاً قطع می‌کنیم
    filter_complex = "[0:a]volume=0[original_audio];"

# اضافه کردن هر سگمنت با زمان‌بندی دقیق
valid_segments = []
for i, sub in enumerate(subs):
    try:
        start_time_ms = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
        filter_complex += f"[{i+1}:a]adelay={start_time_ms}|{start_time_ms}[a{i+1}];"
        valid_segments.append(i)
    except Exception as e:
        print(f"رد کردن سگمنت {i+1} به دلیل مشکل در زمان‌بندی")
        continue

# ترکیب سگمنت‌های معتبر
merge_command = "[original_audio]"
for i in valid_segments:
    merge_command += f"[a{i+1}]"
merge_command += f"amix=inputs={len(valid_segments) + 1}:normalize=0[aout]"
filter_complex += merge_command

# ساخت دستور ffmpeg
input_files = " ".join([f"-i dubbing_project/dubbed_segments/dub_{i+1}.wav" for i in valid_segments])

# استخراج کد زبان از voice_choice برای نام فایل
# این متغیر در سلول قبلی ایجاد شده است
voice_code = voice_choice.split("(")[1].split(")")[0] if "(" in voice_choice else "FA"
output_filename = f'final_dubbed_video_{voice_code}.mp4'

# اجرای دستور نهایی
command = f'ffmpeg -y -i input_video.mp4 {input_files} -filter_complex "{filter_complex}" -map 0:v -map "[aout]" -c:v copy {output_filename}'
subprocess.run(command, shell=True)

if os.path.exists(output_filename):
    # پیام موفقیت بر اساس تنظیمات صدای اصلی
    if keep_original_audio:
        print(f"ویدیوی نهایی با صدای {voice_choice} و حفظ صدای اصلی با میزان {original_audio_volume*100}% با موفقیت ساخته شد!")
    else:
        print(f"ویدیوی نهایی با صدای {voice_choice} (بدون صدای اصلی) با موفقیت ساخته شد!")
    from google.colab import files
    files.download(output_filename)
else:
    print("خطا در ساخت ویدیو. لطفاً موارد زیر را بررسی کنید:")
    print("1. وجود ویدیوی اصلی")
    print("2. وجود تمام فایل‌های صوتی در پوشه dubbing_project/dubbed_segments")

In [1]:
#@title پاکسازی فایل‌های جلسه قبلی
!rm -rf /content/*